# $n$-FullAdder no Qiskit

## 1° Parte: Iniciar e Configurar o backend e as credenciais

In [1]:
# Instalando os requisitos
# Não é necessário caso o env já tenha as bibliotecas
%pip install pylatexenc
%pip install qiskit
%pip install qiskit-ibmq-provider
%pip install qiskit_aer
%pip install qiskit_ibm_runtime
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

  Using cached qiskit_ibmq_provider-0.20.2-py3-none-any.whl.metadata (14 kB)
  Using cached qiskit_terra-0.46.3-cp38-abi3-win_amd64.whl.metadata (13 kB)
  Using cached requests_ntlm-1.1.0-py2.py3-none-any.whl.metadata (938 bytes)
  Using cached numpy-1.23.5.tar.gz (10.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "c:\Users\joseg\anaconda3\envs\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\joseg\anaconda3\envs\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\joseg\anaconda3\envs\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "c:\Users\joseg\anaconda3\envs\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 77, in


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import qiskit
qiskit.__version__

'1.2.4'

In [3]:
# Importando as bibliotecas

from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# Parametros da função 'QiskitRuntimeService'
# channel -> especifica qual canal de serviço está sendo utilizado, No caso, 'ibm_quantum' significa que você está se conectando ao IBM Quantum, o serviço de computação quântica em nuvem da IBM.
# token -> seu token de acesso 

token_str = ''
service = QiskitRuntimeService(channel='ibm_quantum', # cloud ibm
                               token=token_str)

In [5]:
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token=token_str,
    set_as_default=True,
    # Use `overwrite=True` if you're updating your token.
    overwrite=True,
)
 
# Load saved credentials
service = QiskitRuntimeService()

In [6]:
# Conectando ao backend

backend = service.backend(name= "ibm_brisbane")

# 2° Parte: Criar, Simular e Executar

### 1. Método Clássico:

![Full Adder 4 bits](data/full_adder_4bits.png)

Um full adder de 4 bits é um circuito lógico que realiza a adição de dois números binários de 4 bits, levando em conta o carry in (vai um) da posição anterior e gerando um carry out para a próxima posição. Ele é composto por quatro full adders simples, cada um somando um par de bits correspondentes dos números de entrada e o carry in gerado pelo bit anterior.

### 2. Método Quântico

